In [21]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time

# Configurar Chrome en modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

driver.get("https://camara.cl/sala/default.aspx#!/verTabla/4454/958/null/Default/0.1696912497075076")
time.sleep(3)  # Esperar que cargue la página

# Obtener selector de legislaturas
legislaturas = Select(driver.find_element("id", "Legislatura"))

# Iterar por legislaturas desde 370 (mar.2022-mar.2023) hasta 373 (mar.2025-mar.2026)
for i in range(len(legislaturas.options)):
    legislatura_text = legislaturas.options[i].text
    if "370" in legislatura_text or "371" in legislatura_text or "372" in legislatura_text or "373" in legislatura_text:
        legislaturas.select_by_index(i)
        print(f"=== Legislatura {legislatura_text} ===")
        time.sleep(2)  # Esperar que Angular cargue sesiones

        # Obtener selector de sesiones
        sesiones = Select(driver.find_element("id", "Sesion"))

        # Iterar por sesiones
        for j in range(len(sesiones.options)):
            sesion_text = sesiones.options[j].text
            print(f"  Sesión: {sesion_text}")

driver.quit()

=== Legislatura 373ª LEGISLATURA (mar. 2025 - mar. 2026) ===
  Sesión: 72ª, especial, 17 septiembre , 2025
  Sesión: 71ª, ordinaria, 16 septiembre , 2025
  Sesión: 70ª, ordinaria, 15 septiembre , 2025
  Sesión: 69ª, ordinaria, 10 septiembre , 2025
  Sesión: 68ª, ordinaria, 9 septiembre , 2025
  Sesión: 67ª, ordinaria, 8 septiembre , 2025
  Sesión: 66ª, ordinaria, 3 septiembre , 2025
  Sesión: 65ª, especial, 2 septiembre , 2025
  Sesión: 64ª, ordinaria, 2 septiembre , 2025
  Sesión: 63ª, ordinaria, 1 septiembre , 2025
  Sesión: 62ª, ordinaria, 21 agosto , 2025
  Sesión: 61ª, ordinaria, 20 agosto , 2025
  Sesión: 60ª, ordinaria, 19 agosto , 2025
  Sesión: 59ª, ordinaria, 13 agosto , 2025
  Sesión: 58ª, ordinaria, 12 agosto , 2025
  Sesión: 57ª, ordinaria, 11 agosto , 2025
  Sesión: 56ª, ordinaria, 6 agosto , 2025
  Sesión: 55ª, ordinaria, 5 agosto , 2025
  Sesión: 54ª, ordinaria, 4 agosto , 2025
  Sesión: 53ª, ordinaria, 23 julio , 2025
  Sesión: 52ª, ordinaria, 22 julio , 2025
  Sesión:

In [3]:
import os
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from urllib.parse import urlparse, parse_qs, unquote, urljoin

# === CONFIGURACIÓN ===
output_file = os.path.join(os.getcwd(), "pdf_links.txt")

chrome_options = Options()
chrome_options.add_argument("--headless=new")
prefs = {"plugins.always_open_pdf_externally": True}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

# === SCRIPT PRINCIPAL ===
driver.get("https://camara.cl/sala/default.aspx#!/verTabla/4454/958/null/Default/0.1696912497075076")
time.sleep(3)

legislaturas = Select(driver.find_element(By.ID, "Legislatura"))
all_links = []

for i in range(len(legislaturas.options)):
    legislatura_text = legislaturas.options[i].text
    if any(str(x) in legislatura_text for x in range(370, 374)):
        legislaturas.select_by_index(i)
        print(f"\n=== Legislatura {legislatura_text} ===")
        time.sleep(2)

        sesiones = Select(driver.find_element(By.ID, "Sesion"))

        for j in range(len(sesiones.options)):
            sesion_text = sesiones.options[j].text.replace("/", "-").replace(" ", "_")
            print(f"  Sesión: {sesion_text}")
            sesiones.select_by_index(j)
            time.sleep(2)

            try:
                boletin_button = driver.find_element(By.XPATH, "//div[contains(@class,'button-boletin')]")
                if "button-disabled" in boletin_button.get_attribute("class"):
                    print("    ⚠️ Boletín no disponible (botón deshabilitado)")
                    continue

                try:
                    boletin_button.click()
                except ElementNotInteractableException:
                    driver.execute_script("arguments[0].click();", boletin_button)

                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe")))
                visor_iframe = driver.find_element(By.CSS_SELECTOR, "iframe")
                visor_url = visor_iframe.get_attribute("src")
                print(f"    → URL visor: {visor_url}")

                # Extraer el parámetro "file"
                parsed = urlparse(visor_url)
                q = parse_qs(parsed.query)
                if "file" in q:
                    raw_file = unquote(q["file"][0])  # decodificar %2F etc.
                    pdf_url = urljoin("https://camara.cl/sala/Scripts/PDFJs/web/", raw_file)
                    print(f"    ✔ PDF directo: {pdf_url}")
                    all_links.append(f"{legislatura_text.strip()}_{sesion_text} | {pdf_url}")
                else:
                    print("    ⚠️ No se encontró parámetro 'file' en visor")

            except NoSuchElementException:
                print("    ⚠️ No se encontró botón de boletín en esta sesión")
            except Exception as e:
                print(f"    ⚠️ Error inesperado: {e}")

# === GUARDAR LINKS EN TXT ===
with open(output_file, "w", encoding="utf-8") as f:
    for link in all_links:
        f.write(link + "\n")

print(f"\n✅ {len(all_links)} enlaces guardados en {output_file}")
driver.quit()


=== Legislatura 373ª LEGISLATURA (mar. 2025 - mar. 2026) ===
  Sesión: 72ª,_especial,_17_septiembre_,_2025
    ⚠️ Boletín no disponible (botón deshabilitado)
  Sesión: 71ª,_ordinaria,_16_septiembre_,_2025
    ⚠️ Boletín no disponible (botón deshabilitado)
  Sesión: 70ª,_ordinaria,_15_septiembre_,_2025
    ⚠️ Boletín no disponible (botón deshabilitado)
  Sesión: 69ª,_ordinaria,_10_septiembre_,_2025
    ⚠️ Boletín no disponible (botón deshabilitado)
  Sesión: 68ª,_ordinaria,_9_septiembre_,_2025
    → URL visor: https://camara.cl/sala/Scripts/PDFJs/web/viewer.html?file=..%2F..%2F..%2FdoGet.asmx%2FgetDocumentoSesion%3FprmDocumentoId%3D17732%26noCache%3D0.40972254806238395
    ✔ PDF directo: https://camara.cl/sala/doGet.asmx/getDocumentoSesion?prmDocumentoId=17732&noCache=0.40972254806238395
  Sesión: 67ª,_ordinaria,_8_septiembre_,_2025
    → URL visor: https://camara.cl/sala/Scripts/PDFJs/web/viewer.html?file=..%2F..%2F..%2FdoGet.asmx%2FgetDocumentoSesion%3FprmDocumentoId%3D17731%26noCach

In [9]:
import requests
import time
import os
import re

# Ruta al archivo que contiene los nombres y URLs
txt_file = "C:/Users/conjv/Desktop/pdf_links.txt"
# Carpeta donde guardaremos los PDFs
output_folder = "pdfs_descargados"
os.makedirs(output_folder, exist_ok=True)

# Función para limpiar nombres de archivo inválidos
def clean_filename(name):
    return re.sub(r'[^a-zA-Z0-9_\-\.]', '_', name)

# Leer líneas del txt
with open(txt_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

print(f"Se encontraron {len(lines)} documentos para descargar.")

for i, line in enumerate(lines, 1):
    try:
        # Separar nombre y URL por el "|"
        if "|" in line:
            name_part, url = line.split("|", 1)
            name_part = clean_filename(name_part.strip())
        else:
            name_part = f"documento_{i}"
            url = line.strip()

        print(f"\n[{i}/{len(lines)}] Descargando: {name_part}")
        response = requests.get(url.strip(), stream=True, timeout=60)

        if response.status_code == 200:
            # Guardar PDF con nombre de sesión
            filename = os.path.join(output_folder, f"{name_part}.pdf")
            with open(filename, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)

            print(f"  ✔ Guardado como: {filename}")
        else:
            print(f"  ✘ Error HTTP {response.status_code}")

    except Exception as e:
        print(f"  ✘ Error al descargar: {e}")

    # Esperar 10 segundos antes del siguiente
    print("  ⏳ Esperando 10 segundos antes del siguiente...")
    time.sleep(10)

print("\n✅ Descargas finalizadas.")

Se encontraron 482 documentos para descargar.

[1/482] Descargando: 373__LEGISLATURA__mar._2025_-_mar._2026__68___ordinaria__9_septiembre___2025
  ✔ Guardado como: pdfs_descargados\373__LEGISLATURA__mar._2025_-_mar._2026__68___ordinaria__9_septiembre___2025.pdf
  ⏳ Esperando 10 segundos antes del siguiente...

[2/482] Descargando: 373__LEGISLATURA__mar._2025_-_mar._2026__67___ordinaria__8_septiembre___2025
  ✔ Guardado como: pdfs_descargados\373__LEGISLATURA__mar._2025_-_mar._2026__67___ordinaria__8_septiembre___2025.pdf
  ⏳ Esperando 10 segundos antes del siguiente...

[3/482] Descargando: 373__LEGISLATURA__mar._2025_-_mar._2026__66___ordinaria__3_septiembre___2025
  ✔ Guardado como: pdfs_descargados\373__LEGISLATURA__mar._2025_-_mar._2026__66___ordinaria__3_septiembre___2025.pdf
  ⏳ Esperando 10 segundos antes del siguiente...

[4/482] Descargando: 373__LEGISLATURA__mar._2025_-_mar._2026__65___especial__2_septiembre___2025
  ✔ Guardado como: pdfs_descargados\373__LEGISLATURA__mar._20